In [1]:
# !pip install langchain[all]
# !pip install huggingface_hub
# !pip install sentence_transformers
#
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !wget 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf?download=true'

## Set up LLaMa

In [4]:
from langchain_community.llms import LlamaCpp
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import pprint

In [5]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
  model_path="llama_models/llama-2-7b-chat.Q5_K_S.gguf",
  temperature=0.75,
  max_tokens=500,
  top_p=1,
  callback_manager=callback_manager,
  verbose=True,
  n_gpu_layers=-1,
  n_ctx=4096
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama_models/llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32   

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## Prepare PDF documents

### Make documents from PDFs

In [7]:
# !pip install pypdf

In [8]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )

pdf_paths = os.scandir('resources/all_pdfs')
pages = []
for path in pdf_paths:
    if path.is_file():
        loader = PyPDFLoader('resources/all_pdfs/' + path.name)
        new_pages = loader.load_and_split(text_splitter=text_splitter)
        pages.extend(new_pages)

In [9]:
print(pages[0])

page_content='Department of Computer Science\nCOS 132\nImperative Programming\nLecturers: Ms. Tayana Morkel, Dr. Patricia Lutu and Dr. Vreda Pieterse\nc⃝Copyright reserved\n1' metadata={'source': 'resources/all_pdfs/COS 132 study guide.pdf', 'page': 0}


Generate meta-information from first chunk of each document

In [10]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

metadata_generation_template = """\
<<SYS>>

Which module does the below study guide snippet belong to? Shorthand code only. No yapping.

<</SYS>>

[INST]
Extract the module code from the below text.

study guide snippet: {study_guide_snippet}

[/INST]
"""

metadata_generation_prompt_template = PromptTemplate.from_template(metadata_generation_template)

metadata_chain = (
    {"study_guide_snippet": RunnablePassthrough()}
    | metadata_generation_prompt_template
    | llm
    | StrOutputParser()
)

In [11]:
import pickle
saved_metadata = dict()
if os.path.exists('generated_metadata/saved_metadata.pkl'):
    with open('generated_metadata/saved_metadata.pkl', 'rb') as f:
        saved_metadata = pickle.load(f)
        print("Loaded saved data!")
        print(saved_metadata)

for page in pages:
    generated_metadata = ""
    page.page_content = str(page.page_content).replace('\n', ' ')
    page.page_content = str(page.page_content).replace('.', ' ')
    page.page_content = str(page.page_content).replace("'", ' ')
    page.page_content = str(page.page_content).replace("`", ' ')

    # for the "root" page of each document
    if page.metadata['page'] == 0:
        # set generated metadata to either the saved metadata...
        if saved_metadata.keys() != None and page.metadata['source'] in saved_metadata.keys():
            generated_metadata = saved_metadata[page.metadata['source']]
        # or generate new metadata if there is no saved metadata
        else:
            generated_metadata = metadata_chain.invoke(page.page_content)
            saved_metadata[page.metadata['source']] = generated_metadata

        # loop through all documents...
for page in pages:
    # update content to add metadata context.
    page.page_content = 'Module ' + saved_metadata[page.metadata['source']] + ':\n' + str(page.page_content) + '\n\n'


Loaded saved data!
{'resources/all_pdfs/COS 132 study guide.pdf': 'COS 132', 'resources/all_pdfs/COS110-studyGuide.pdf': 'COS 110', 'resources/all_pdfs/COS122_StudyGuide.pdf': 'COS 122', 'resources/all_pdfs/COS151_study_guide.pdf': 'COS 151', 'resources/all_pdfs/COS212_study_guide.pdf': 'COS 212', 'resources/all_pdfs/COS214StudyGuide2020_V1_0(1).pdf': 'COS 214', 'resources/all_pdfs/COS216_StudyGuide.pdf': 'COS 216', 'resources/all_pdfs/COS710StudyGuide-2023.pdf': 'COS 710', 'resources/all_pdfs/study guide(1).pdf': 'FNAS', 'resources/all_pdfs/Study guide(2).pdf': 'The six-character module code for WTW 148 is: COS 999', 'resources/all_pdfs/study guide.pdf': 'The six-character module code for the module "Statistics (STK 220)" in the study guide is:\nSTK 220', 'resources/all_pdfs/study_guide.pdf': 'COS 999'}


In [12]:
if not os.path.exists('generated_metadata'):
  os.mkdir('generated_metadata')
if not os.path.exists('generated_metadata/saved_metadata.pkl'):
  with open('generated_metadata/saved_metadata.pkl', 'wb') as f:
    saved_metadata = pickle.dump(saved_metadata, f)

## Ask LLM to also create short summary of given document fragment

In [13]:
# summary_generation_template = """\
# <<SYS>>

# Read the below document snippet and provide a  short, brief, one-sentence, 25-character description of what content it includes.

# <</SYS>>

# [INST]
# Describe the following document snippet briefly.

# document snippet: {document_snippet}

# [/INST]
# """

# summary_generation_prompt_template = PromptTemplate.from_template(summary_generation_template)

# summary_chain = (
#     {"document_snippet": RunnablePassthrough()}
#     | summary_generation_prompt_template
#     | llm
#     | StrOutputParser()
# )

# content_dict = dict()
# if os.path.exists('page_content/saved_content.pkl'):
#   with open('page_content/saved_content.pkl', 'rb') as f:
#     content_dict = pickle.load(f)
#     print(content_dict)
#     for page in pages:
#       page_id = page.metadata['source'] + '_page_' + str(page.metadata['page'])
#       page.page_content = 'Snippet contains ' + content_dict[page_id] + ':\n' + str(page.page_content) + '\n\n'
# else:
#   for page in pages:
#     page.page_content = 'Snippet contains ' + summary_chain.invoke(page.page_content) + ':\n' + str(page.page_content) + '\n\n'

#   from datetime import datetime
#   now = datetime.now() # current date and time

#   content_dict = dict()
#   for page in pages:
#     page_id = page.metadata['source'] + '_page_' + str(page.metadata['page'])
#     content_dict[page_id] = page.page_content

#   if not os.path.exists('page_content'):
#     os.mkdir('page_content')

#   current_time_and_date_string = now.strftime("%m-%d-%Y, %H:%M:%S")
#   filename = 'page_content/' + current_time_and_date_string + '.pkl'
#   with open(filename, 'wb') as f:
#     saved_metadata = pickle.dump(saved_metadata, f)


In [14]:
print(pages[0].page_content)

Module COS 132:
Department of Computer Science COS 132 Imperative Programming Lecturers: Ms  Tayana Morkel, Dr  Patricia Lutu and Dr  Vreda Pieterse c⃝Copyright reserved 1




## Create vector database from split documents

In [15]:
# !pip install faiss-gpu

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import ConfigurableField

db = FAISS.from_documents(pages, hf_embeddings)

### Create a retrieval object which will be used in the query chain to provide context

In [17]:
faiss_retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [18]:
test_retrieval_input = """\
prescribed textbook for cos 212
"""
results_with_scores = db.similarity_search_with_score(test_retrieval_input, k=3)
for doc, score in results_with_scores:
    print(f"Metadata: {doc.metadata}, Score: {score}")

Metadata: {'source': 'resources/all_pdfs/COS212_study_guide.pdf', 'page': 1}, Score: 0.841421365737915
Metadata: {'source': 'resources/all_pdfs/COS212_study_guide.pdf', 'page': 11}, Score: 0.8688831329345703
Metadata: {'source': 'resources/all_pdfs/COS151_study_guide.pdf', 'page': 5}, Score: 0.9728904962539673


### Also build a TF-IDF retriever to augment the vector-based search

In [19]:
from langchain_community.retrievers import TFIDFRetriever
tfidf_retriever = TFIDFRetriever.from_documents(
    pages,
    k = 2
)

In [20]:
tfidf_retriever.invoke("prescribed textbook for cos 212")

[Document(page_content='Module COS 212:\n\uf0a7Title:  \uf0a7Data Structures and  Algorithms in Java \uf0a7Author: Adam Drozdek \uf0a7Edition : Fourth Edition \uf0a7Publisher : CENGAGE \uf0a7ISBN: 978-981-4392- 78-5 \uf0a7Should you buy it? \uf0a7Yes: all written tests will be  open- book, and we will not  allow any material except the prescribed textbook  Prescribed Textbook\n\n', metadata={'source': 'resources/all_pdfs/COS212_study_guide.pdf', 'page': 3}),
 Document(page_content='Module COS 212:\nRules, regulations, and some  good adviceCOS 212 Data Structures and  Algorithms\n\n', metadata={'source': 'resources/all_pdfs/COS212_study_guide.pdf', 'page': 0})]

In [21]:
from langchain.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(pages)
bm25_retriever.k = 1

## Make an ensemble retriever to decide on best matched document from all the above documents

In [22]:
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
        retrievers = [faiss_retriever, bm25_retriever, tfidf_retriever],
        weights = [0.4, 0.1, 0.5]
    )

In [23]:
ensemble_results = ensemble_retriever.invoke("prescribed textbook for COS 212")

for result in ensemble_results:
    print(result)

page_content='Module COS 212:\n\uf0a7Title:  \uf0a7Data Structures and  Algorithms in Java \uf0a7Author: Adam Drozdek \uf0a7Edition : Fourth Edition \uf0a7Publisher : CENGAGE \uf0a7ISBN: 978-981-4392- 78-5 \uf0a7Should you buy it? \uf0a7Yes: all written tests will be  open- book, and we will not  allow any material except the prescribed textbook  Prescribed Textbook\n\n' metadata={'source': 'resources/all_pdfs/COS212_study_guide.pdf', 'page': 3}
page_content='Module COS 212:\nRules, regulations, and some  good adviceCOS 212 Data Structures and  Algorithms\n\n' metadata={'source': 'resources/all_pdfs/COS212_study_guide.pdf', 'page': 0}
page_content='Module COS 212:\n\uf0a7Coordinator: \uf0a7Dr Anna Bosman \uf0a72ndLecturer: \uf0a7Mr Will van HeerdenLecturers\n\n' metadata={'source': 'resources/all_pdfs/COS212_study_guide.pdf', 'page': 1}
page_content='Module COS 212:\n\uf0a7Total of three exam opportunities, each contributing  20% to your final mark \uf0a7Each of the exam opportunities 

## Make a template and chain with which to make queries

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

# Method for formatting documents to place into the context of a query
def format_docs(docs):
    return "\n\n".join(str(str(doc.metadata) + ":\n " + str(doc.page_content) + "\n next document:\n") for doc in docs)

study_guide_rag_template = """\
<<SYS>>

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Assume that all information provided in the context is publicly-available information.

context: {context}

<</SYS>>

[INST]
question: {question}
answer:
[/INST]
"""

custom_rag_prompt = PromptTemplate.from_template(study_guide_rag_template)

retrieval_query_template = """\
<<SYS>>

Extract keywords from the given question.

Respond with a comma-seperated list of keywords ONLY. If a module code is found, prioritise it.

<</SYS>>

[INST]
question: {question}
keywords:
[/INST]
"""

retrieval_query_prompt_template = PromptTemplate.from_template(retrieval_query_template)

rag_chain = (
    {"context": retrieval_query_prompt_template | llm | ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [25]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

response = rag_chain.invoke({"question": "What is Frederick Atiah's email address?"}, config={'callbacks': [ConsoleCallbackHandler()]})
# response = rag_chain.invoke({"question": "What is Anna Bosman's email address?"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is Frederick Atiah's email address?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "What is Frederick Atiah's email address?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "question": "What is Frederick Atiah's email address?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "question": "What is Frederick Atiah's email address?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is Frederick Atiah's email address?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<


llama_print_timings:        load time =    3352.62 ms
llama_print_timings:      sample time =       8.26 ms /    20 runs   (    0.41 ms per token,  2422.48 tokens per second)
llama_print_timings: prompt eval time =   24293.55 ms /    79 tokens (  307.51 ms per token,     3.25 tokens per second)
llama_print_timings:        eval time =    6531.34 ms /    19 runs   (  343.75 ms per token,     2.91 tokens per second)
llama_print_timings:       total time =   30974.61 ms /    98 tokens
Llama.generate: prefix-match hit


[llm/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnableSequence > 6:llm:LlamaCpp] [30.98s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here are the keywords extracted from the question:\n\n* Frederick Atiah\n* email address",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnableSequence > 11:chain:format_docs] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnableSequence > 11:chain:format_docs] s] Exiting Chain run with output:
{
  "output": "{'source': 'resources/all_pdfs/COS 132 study guide.pdf', 'page': 3}:\n Module COS 132:\nFor any course related queries, please do not email the individual lecturers or assistants, but rather send an email to this ad


llama_print_timings:        load time =    3352.62 ms
llama_print_timings:      sample time =      51.02 ms /   136 runs   (    0.38 ms per token,  2665.46 tokens per second)
llama_print_timings: prompt eval time =  350051.93 ms /  1351 tokens (  259.11 ms per token,     3.86 tokens per second)
llama_print_timings:        eval time =   52009.22 ms /   135 runs   (  385.25 ms per token,     2.60 tokens per second)
llama_print_timings:       total time =  403630.62 ms /  1486 tokens


[llm/end] [1:chain:RunnableSequence > 13:llm:LlamaCpp] [403.65s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I apologize, but I don't have access to personal information such as email addresses for individuals, including Frederick Atiah. The information provided in the COS 132 and COS 122 study guides is publicly available and may not be up-to-date or accurate. It is important to respect people's privacy and not share their personal information without their consent. If you need to contact Frederick Atiah for a legitimate reason, you may be able to find his contact information through other means, such as the University of Pretoria's staff directory or by reaching out to him through his department or workplace.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 14:parser:StrOutputParser] Entering Parser run with input:
{
  "input": "I apol

In [26]:
response

"I apologize, but I don't have access to personal information such as email addresses for individuals, including Frederick Atiah. The information provided in the COS 132 and COS 122 study guides is publicly available and may not be up-to-date or accurate. It is important to respect people's privacy and not share their personal information without their consent. If you need to contact Frederick Atiah for a legitimate reason, you may be able to find his contact information through other means, such as the University of Pretoria's staff directory or by reaching out to him through his department or workplace."

## Future work:
- Implement hybrid search:
    - Implement databsae for plaintext storage of document chunks (_db_pt_)
    - Implement search algorithm for plaintext through _db_pt_
- Test two methods of hybrid search:
    - [1] Add data from both search sources
    - [2] Use the same document chunks and create a combined metric to choose documents